# 3주차 스터디

Made by 35기 코딩부장 류제현

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## 1. Pandas 더 친숙해지기

    실제 데이터 전처리 과정을 함께 따라해 봅시다


* 코스닥 상장 기업들의 년도별 전환사채(CB)발행 기록이 담긴 3개의 CSV파일이 있습니다. 
* 데이터셋을 하나로 합치고, 데이터가 완전한지 확인해보겠습니다

In [2]:
# 우선 데이터셋을 불러오겠습니다
df2019 = pd.read_csv("전환사채권발행결정_코스닥_2019-01-01-2019-12-31.csv",index_col=0)
df2020 = pd.read_csv("전환사채권발행결정_코스닥_2020-01-01-2020-12-31.csv",index_col=0)
df2021 = pd.read_csv("전환사채권발행결정_코스닥_2021-01-01-2021-12-31.csv",index_col=0)

# 아, 그런데 만약 년도가 2000년부터 2020년까지 엄청 많으면 이렇게 코드를 복사하기 귀찮지 않을까요?
# -> CSV파일 이름이 유사한데 반복문으로 해결할 수 있지 않을까요?
for year in range(2019,2022):
    globals()["df"+str(year)] = pd.read_csv("전환사채권발행결정_코스닥_{}-01-01-{}-12-31.csv".format(year,year), index_col=0)

In [3]:
# 확인해 볼까요?
df2019.head()

,회사명,기업공시코드,공시일,종류,총액(원),사채만기일,표면이자율(%),만기이자율(%),사채발행방법,공시코드
0,네오티스,85910,2019-12-27,무기명식 이권부 무보증 사모 전환사채,"10,000,000,000",2025년 01월 03일,0.0,1.0,사모,20191227000693
1,지나인제약,78650,2019-12-27,무기명식 이권부 무보증 사모 전환사채,"8,000,000,000",2025년 01월 03일,0.0,0.0,사모,20191227000514
2,화신테크,86250,2019-12-27,무기명식 이권부 무보증 사모 전환사채,"20,000,000,000",2023년 03월 17일,5.0,5.0,사모,20191227000010
3,테크엘,64520,2019-12-27,무기명 사모 일반 전환사채,"5,000,000,000",2024년 12월 26일,2.0,2.0,사모,20191227000006
4,티로보틱스,117730,2019-12-26,무기명식 무보증 사모 전환사채,"10,000,000,000",2022년 12월 27일,0.0,0.0,사모,20191226000394


In [4]:
# 2019,2020,2021데이터셋을 하나로 합칩시다! 
all_df = pd.concat([df2021,pd.concat([df2020,df2019])])

# 합친 후, 공시일 순으로 정렬해 봅시다
all_df = all_df.sort_values(by="공시일").reset_index(drop=True)
all_df

,회사명,기업공시코드,공시일,종류,총액(원),사채만기일,표면이자율(%),만기이자율(%),사채발행방법,공시코드
0,대창솔루션,96350,2019-01-03,무기명식 담보부 사모전환사채,"35,000,000,000",2022년 03월 31일,0.0,4.5,사모,20190103000744
1,리더스 기술투자,19570,2019-01-04,무기명식 이권부 무보증 사모 전환사채,"10,000,000,000",2022년 02월 25일,0,3,사모,20190104000909
2,리더스 기술투자,19570,2019-01-04,무기명식 이권부 무보증 사모 전환사채,"5,000,000,000",2022년 02월 25일,0,3,사모,20190104000906
3,엔에스엔,31860,2019-01-04,무기명식 무보증 이권부 사모 전환사채,"3,000,000,000",2022년 01월 18일,3,5,사모,20190104000863
4,장원테크,174880,2019-01-04,무기명식 이권부 무보증 사모 전환사채,"20,000,000,000",2022년 02월 27일,2.0,2.0,사모,20190104000871
...,...,...,...,...,...,...,...,...,...,...
1368,이원컴포텍,88290,2021-12-21,무기명식 이권부 무보증 사모 전환사채,"15,000,000,000",2025년 01월 25일,2,5,사모,20211221000754
1369,옵트론텍,82210,2021-12-24,무기명식 이권부 무보증 사모 전환사채,"1,600,000,000",2024년 12월 30일,0.00,0.00,사모,20211224000603
1370,노터스,278650,2021-12-27,무기명식 이권부 무보증 사모 전환사채,"54,600,000,000",2025년 01월 13일,0.0,1.0,사모,20211227000668
1371,노터스,278650,2021-12-27,무기명식 이권부 무보증 사모 전환사채,"40,000,000,000",2025년 01월 13일,0.0,1.0,사모,20211227000669


    본격적인 데이터 분석에 앞서, 데이터에 이상한 값이 들어있는지 확인해봐야 합니다

이 데이터는 제가 직접 코드를 짜서 기업공시사이트(KIND)에서 크롤링해 왔습니다. 따라서, 어떤 방식으로 데이터를 수집했는지에 대한 지식이 있습니다

데이터셋의 ["회사명","기업공시코드","공시일"] 컬럼에는 이상한 데이터가 들어있을 확률이 0%입니다.

따라서 저에게 필요한 정보인 "총액(원)", "표면이자율(%)","만기이자율(%)"에 이상한 값이 있는지 살펴보겠습니다

In [5]:
# 우선 "총액(원)" 컬럼을 살펴보겠습니다
## 만약에 이 컬럼에 이상한 값 (숫자가 아닌 문자, 소수 등)이 들어있다면 이 데이터 타입이 int형으로 안바뀌지 않을까요?
## -> 이 아이디어에 착안해서 총액(원)의 데이터타입을 int로 변환해 보겠습니다

# 숫자 중간에 쉼표가 있기 때문에 우선 제거해 줍니다
all_df["총액(원)"] = all_df["총액(원)"].str.replace(",","")

# 이제 int형으로 변형해줍니다
all_df["총액(원)"] = all_df["총액(원)"].astype("int64")
# 오 잘 바뀌네요! 그러면 여기엔 문자는 없고, 정수값만 있다는 뜻 입니다 (문자가 포함되어 있으면 에러메시지가 뜹니다. 궁금하면 직접확인!)

In [6]:
# 그러면 이제 총액(원)에 0원, 10원 등과 같은 말도 안되는 숫자가 있는 지 확인해 보겠습니다
all_df.loc[all_df["총액(원)"] <= 1000]
# 최소 4억 이상이네요! 충분이 납득이 가는 숫자입니다! 총액 컬럼 확인을 이제 마치겠습니다

,회사명,기업공시코드,공시일,종류,총액(원),사채만기일,표면이자율(%),만기이자율(%),사채발행방법,공시코드


In [7]:
# 다음으로 표면이자울(%)를 확인해보겠습니다
## 소수로 표시되기 때문에, 우선 float로 변환해보겠습니다
all_df["표면이자율(%)"] = all_df["표면이자율(%)"].astype("float64")

ValueError: could not convert string to float: '-'

In [8]:
# 이런 오류가 떴네요! "could not convert string to float: '-'" string값이 있다는 뜻이네요.. 한번 살펴볼까요?
all_df.loc[all_df["표면이자율(%)"] == "-"]

,회사명,기업공시코드,공시일,종류,총액(원),사채만기일,표면이자율(%),만기이자율(%),사채발행방법,공시코드
6,옵트론텍,82210,2019-01-07,무기명식 이권부 무보증 사모 전환사채,5000000000,2024년 01월 09일,-,-,사모,20190107000666
52,데코앤이,17680,2019-02-14,무기명식 무보증 사모 전환사채,850000000,2022년 02월 14일,-,3,사모,20190214000757
64,이엠네트웍스,87730,2019-02-22,무기명식 이권부무보증 사모 전환사채,2000000000,2022년 02월 22일,-,5,사모,20190222000001
213,에스와이,109610,2019-05-31,무기명식 이권부 무보증 무담보 사모 전환사채,5000000000,2024년 06월 04일,-,-,사모,20190531000634
306,셀루메드,49180,2019-08-06,무기명식 이권부 무보증 사모 전환사채,6325000500,2022년 08월 07일,-,3,사모,20190806000612
312,퓨전,195440,2019-08-14,무기명식 이권부 무보증 사모 전환사채,2500000000,2022.08.14,-,4.0,사모,20190814002879
371,버킷스튜디오,66410,2019-10-02,무기명식 무보증 사모 전환사채,5000000000,2022년 10월 11일,-,3.0,사모,20191002000478
435,명성티엔에스,257370,2019-12-06,무기명식 이권부 무보증 사모 전환사채,5000000000,2023.02.19,-,2,사모,20191206000337
463,제넨바이오,72520,2020-01-07,무기명식 이권부무보증 사모 전환사채,7000000000,2023.01.07,-,3,사모,20200107000385
491,아리온,58220,2020-02-21,무기명식 이권부 무보증 사모 전환사채,3499000000,2023년 02월 21일,-,-,사모,20200221000683


In [9]:
# 데이터분석 진행을 위해서 '-'값에 0을 넣어주겠습니다
# 실제 분석 과정에서는 직접 사이트에 들어가 공시 값들을 확인해야 합니다
all_df["표면이자율(%)"] = all_df["표면이자율(%)"].str.replace("-","0")
all_df["만기이자율(%)"] = all_df["만기이자율(%)"].str.replace("-","0")

In [10]:
# 다시 float로 변환해볼까요?
all_df["표면이자율(%)"] = all_df["표면이자율(%)"].astype("float64")

ValueError: could not convert string to float: '0.0, '

In [11]:
# 이런... 0.0, 라는 값이 있어서 (쉼표때문에) 변환이 안되는 군요... 쉼표를 제거합시다
all_df["표면이자율(%)"] = all_df["표면이자율(%)"].str.replace("," , "")

In [12]:
# 다시 float로 변환해 봅시다
all_df["표면이자율(%)"] = all_df["표면이자율(%)"].astype("float64")
# 드디어 성공했습니다!!

In [13]:
# 길고긴 전처리가 끝났습니다!!
all_df

,회사명,기업공시코드,공시일,종류,총액(원),사채만기일,표면이자율(%),만기이자율(%),사채발행방법,공시코드
0,대창솔루션,96350,2019-01-03,무기명식 담보부 사모전환사채,35000000000,2022년 03월 31일,0.0,4.5,사모,20190103000744
1,리더스 기술투자,19570,2019-01-04,무기명식 이권부 무보증 사모 전환사채,10000000000,2022년 02월 25일,0.0,3,사모,20190104000909
2,리더스 기술투자,19570,2019-01-04,무기명식 이권부 무보증 사모 전환사채,5000000000,2022년 02월 25일,0.0,3,사모,20190104000906
3,엔에스엔,31860,2019-01-04,무기명식 무보증 이권부 사모 전환사채,3000000000,2022년 01월 18일,3.0,5,사모,20190104000863
4,장원테크,174880,2019-01-04,무기명식 이권부 무보증 사모 전환사채,20000000000,2022년 02월 27일,2.0,2.0,사모,20190104000871
...,...,...,...,...,...,...,...,...,...,...
1368,이원컴포텍,88290,2021-12-21,무기명식 이권부 무보증 사모 전환사채,15000000000,2025년 01월 25일,2.0,5,사모,20211221000754
1369,옵트론텍,82210,2021-12-24,무기명식 이권부 무보증 사모 전환사채,1600000000,2024년 12월 30일,0.0,0.00,사모,20211224000603
1370,노터스,278650,2021-12-27,무기명식 이권부 무보증 사모 전환사채,54600000000,2025년 01월 13일,0.0,1.0,사모,20211227000668
1371,노터스,278650,2021-12-27,무기명식 이권부 무보증 사모 전환사채,40000000000,2025년 01월 13일,0.0,1.0,사모,20211227000669


## 2. 판다스 응용 예시

    각 데이터별로 하나라도 NaN값이 있는 팽귄을 찾아봅시다

In [14]:
from seaborn import load_dataset

df = load_dataset('penguins')
df

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female
...,...,...,...,...,...,...,...
339,Gentoo,Biscoe,NaN,NaN,NaN,NaN,NaN
340,Gentoo,Biscoe,46.8,14.3,215.0,4850.0,Female
341,Gentoo,Biscoe,50.4,15.7,222.0,5750.0,Male
342,Gentoo,Biscoe,45.2,14.8,212.0,5200.0,Female


In [16]:
# 결측치(NaN) 값을 찾아보자 -> nan은 연산자가 먹히지 않는다
#df == np.nan -> 작동하지 않음

df.isna()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False
3,False,False,True,True,True,True,True
4,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...
339,False,False,True,True,True,True,True
340,False,False,False,False,False,False,False
341,False,False,False,False,False,False,False
342,False,False,False,False,False,False,False


In [17]:
# True는 1, False는 0인 성질을 사용

idx = df.isna().sum(axis=1) >=1 
# 이걸 인덱스로 주면 되겠다!!

In [18]:
df[idx]

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
8,Adelie,Torgersen,34.1,18.1,193.0,3475.0,NaN
9,Adelie,Torgersen,42.0,20.2,190.0,4250.0,NaN
10,Adelie,Torgersen,37.8,17.1,186.0,3300.0,NaN
11,Adelie,Torgersen,37.8,17.3,180.0,3700.0,NaN
47,Adelie,Dream,37.5,18.9,179.0,2975.0,NaN
246,Gentoo,Biscoe,44.5,14.3,216.0,4100.0,NaN
286,Gentoo,Biscoe,46.2,14.4,214.0,4650.0,NaN
324,Gentoo,Biscoe,47.3,13.8,216.0,4725.0,NaN
336,Gentoo,Biscoe,44.5,15.7,217.0,4875.0,NaN


In [21]:
# isin을 사용해도 된다
df.isin([np.nan]).any(1)

0      False
1      False
2      False
3       True
4      False
       ...  
339     True
340    False
341    False
342    False
343    False
Length: 344, dtype: bool

    머신러닝 개론(PPT)